<a href="https://colab.research.google.com/github/Emenike-Amara/Projects/blob/main/Data_Exploration_with_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [16]:
#Python's built-in sqlite3 module to create an in-memory SQLite database as I do not a live connection

import sqlite3

conn = sqlite3.connect(':memory:')

In [ ]:
!pip install pandas pandasql

In [43]:
from pandas.io.sql import PandasSQL

In [50]:
#Loading data and declaring path reference
import pandas as pd
from pandasql import sqldf
from google.colab import drive
drive.mount('/content/drive')
path1='/content/drive/My Drive/Features data set.csv'
path2= '/content/drive/My Drive/sales data-set.csv'
path3 = '/content/drive/My Drive/stores data-set.csv'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# Table 1 

feature_data = pd.read_csv(path1)
feature_data.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [61]:
# Table 2 

sales_data = pd.read_csv(path2)
sales_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [51]:
# Table 3

store_data = pd.read_csv(path3)
store_data.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [53]:
feature_data.to_sql('feature_data', conn, index=False)

8190

In [ ]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM feature_data ')
results = cursor.fetchall()
results

In [57]:
#Introducing my SQL Queries

SQLquery = '''
SELECT *
FROM sales_data
'''
result = sqldf(SQLquery)
result

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,0
1,1,1,12/02/2010,46039.49,1
2,1,1,19/02/2010,41595.55,0
3,1,1,26/02/2010,19403.54,0
4,1,1,05/03/2010,21827.90,0
...,...,...,...,...,...
421565,45,98,28/09/2012,508.37,0
421566,45,98,05/10/2012,628.10,0
421567,45,98,12/10/2012,1061.02,0
421568,45,98,19/10/2012,760.01,0
